In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.svm import SVC 
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, accuracy_score
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [ ]:
def drop_irrelevant_columns(data):
    """Drops irrelevant or highly missing columns."""
    drop_cols = ['id', 'member_id', 'url', 'desc', 'next_pymnt_d',
                 'annual_inc_joint', 'dti_joint', 'verification_status_joint',
                 'mths_since_last_major_derog', 'mths_since_last_delinq',
                 'mths_since_last_record', 'tot_coll_amt', 'tot_cur_bal']
    
    cols_to_drop_existing = [col for col in drop_cols if col in data.columns]
    data = data.drop(columns=cols_to_drop_existing)
    print(f"Dropped columns: {cols_to_drop_existing}")
    return data

In [ ]:
def define_target_variable(data):
    """Defines the binary target variable 'loan_default'."""
    closed_statuses = [
        'Fully Paid', 'Charged Off', 'Default',
        'Does not meet the credit policy. Status:Fully Paid',
        'Does not meet the credit policy. Status:Charged Off'
    ]
    
    # Categorize loan status into default (1) or non-default (0)
    data['loan_default'] = data['loan_status'].apply(lambda x: 1 if x in ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off'] else (0 if x in ['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid', 'Current', 'In Grace Period', 'Late (16-30 days)', 'Late (31-120 days)'] else np.nan))
    
    # Drop rows where loan_default could not be determined
    initial_rows = data.shape[0]
    data.dropna(subset=['loan_default'], inplace=True)
    print(f"Dropped {initial_rows - data.shape[0]} rows with undefined 'loan_default' status.")
    
    data['loan_default'] = data['loan_default'].astype(int)
    print("Target variable 'loan_default' created.")
    print(data['loan_default'].value_counts())
    return data

In [ ]:
def handle_missing_values(data):
    """Fills missing values based on column type."""
    for col in data.columns:
        if data[col].dtype == 'object':  # Categorical
            if data[col].isnull().sum() > 0:
                mode_val = data[col].mode()[0]
                data[col].fillna(mode_val, inplace=True)
                # print(f"Filled missing values in '{col}' with mode: {mode_val}")
        else: 
            if data[col].isnull().sum() > 0:
                median_val = data[col].median()
                data[col].fillna(median_val, inplace=True)
                # print(f"Filled missing values in '{col}' with median: {median_val}")
    print("Missing values handled.")
    return data

In [ ]:
def cap_outliers(data, numerical_cols):
    """Caps numerical outliers at the 99th percentile."""
    for col in numerical_cols:
        if col in data.columns:
            upper_bound = data[col].quantile(0.99)
            data[col] = np.where(data[col] > upper_bound, upper_bound, data[col])
            # print(f"Capped outliers in '{col}' at 99th percentile.")
    print("Outliers capped for numerical columns.")
    return data

In [ ]:
def create_financial_ratios(data):
    """Creates loan-to-income and credit utilization ratios."""
    if 'loan_amnt' in data.columns and 'annual_inc' in data.columns:
        data['loan_to_income_ratio'] = data['loan_amnt'] / (data['annual_inc'] + 1e-6)
    else:
        print("Warning: 'loan_amnt' or 'annual_inc' not found. Skipping 'loan_to_income_ratio' calculation.")
        data['loan_to_income_ratio'] = np.nan

    if 'revol_hi_lim' in data.columns and 'revol_bal' in data.columns:
        data['revol_hi_lim'] = data['revol_hi_lim'].replace(0, np.nan)
        data['revol_hi_lim'].fillna(data['revol_hi_lim'].median(), inplace=True)
        data['credit_util_ratio'] = data['revol_bal'] / (data['revol_hi_lim'] + 1e-6)
        print("Financial ratios created.")
    else:
        print("Warning: 'revol_hi_lim' or 'revol_bal' not found. Skipping 'credit_util_ratio' calculation.")
        data['credit_util_ratio'] = np.nan
    
    return data

In [ ]:
def extract_date_features(data):
    """Extracts year and month from date columns and calculates credit history length."""
    date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']
    for col in date_cols:
        # Convert only existing date columns
        if col in data.columns:
            # Handle mixed date formats by coercing errors
            data[col] = pd.to_datetime(data[col], format='%b-%y', errors='coerce') 
            data[f'{col}_month'] = data[col].dt.month
            data[f'{col}_year'] = data[col].dt.year
            # Drop the original date column after extracting features
            data = data.drop(columns=[col])
    
    # Calculate credit history length, handling potential NaNs from date conversion
    if 'issue_d_year' in data.columns and 'earliest_cr_line_year' in data.columns:
        data['credit_history_length'] = data['issue_d_year'] - data['earliest_cr_line_year']
        data['credit_history_length'].fillna(data['credit_history_length'].median(), inplace=True) # Impute any NaNs
    print("Date-based features extracted.")
    return data

In [ ]:
def convert_string_to_numeric(data):
    """Converts string-based numeric columns to actual numeric types."""
    if 'emp_length' in data.columns:
        emp_length_map = {
            '10+ years': 10, '9 years': 9, '8 years': 8, '7 years': 7, '6 years': 6,
            '5 years': 5, '4 years': 4, '3 years': 3, '2 years': 2, '1 year': 1,
            '< 1 year': 0.5, 'n/a': np.nan
        }
        data['emp_length_numeric'] = data['emp_length'].replace(emp_length_map).astype(float)
        data['emp_length_numeric'].fillna(data['emp_length_numeric'].median(), inplace=True)
        data = data.drop(columns=['emp_length'])
    else:
        print("Warning: 'emp_length' not found. Skipping 'emp_length_numeric' conversion.")

    if 'term' in data.columns:
        data['term_numeric'] = data['term'].astype(str).str.extract('(\\d+)').astype(float)
        data = data.drop(columns=['term'])
    else:
        print("Warning: 'term' not found. Skipping 'term_numeric' conversion.")

    if 'int_rate' in data.columns:
        data['int_rate_numeric'] = data['int_rate'].astype(str).str.replace('%', '', regex=False).astype(float)
        data = data.drop(columns=['int_rate'])
    else:
        print("Warning: 'int_rate' not found. Skipping 'int_rate_numeric' conversion.")

    if 'revol_util' in data.columns:
        data['revol_util_numeric'] = data['revol_util'].astype(str).str.replace('%', '', regex=False).astype(float)
        data = data.drop(columns=['revol_util'])
    else:
        print("Warning: 'revol_util' not found. Skipping 'revol_util_numeric' conversion.")
        
    print("String-based numeric features converted.")
    return data

In [ ]:
def identify_feature_types(data, exclude_cols=[]):
    """Identifies numerical and categorical columns."""
    numerical_cols = data.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = data.select_dtypes(include='object').columns.tolist()
    
    # Exclude target variable and any other specified columns
    if 'loan_default' in numerical_cols:
        numerical_cols.remove('loan_default')
    
    numerical_cols = [col for col in numerical_cols if col not in exclude_cols]
    categorical_cols = [col for col in categorical_cols if col not in exclude_cols]

    print(f"Identified {len(numerical_cols)} numerical and {len(categorical_cols)} categorical columns.")
    return numerical_cols, categorical_cols

In [ ]:
def create_preprocessing_pipeline(numerical_cols, categorical_cols):
    """Creates a preprocessing pipeline using ColumnTransformer."""
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
        ],
        remainder='passthrough' 
    )
    print("Preprocessing pipeline created.")
    return preprocessor

In [ ]:
def calculate_woe_iv(data, feature, target):
    """
    Calculates Weight of Evidence (WoE) and Information Value (IV) for a given feature.
    
    Args:
        data (pd.DataFrame): The DataFrame containing the feature and target columns.
        feature (str): The name of the feature column.
        target (str): The name of the binary target column (0s and 1s).
        
    Returns:
        tuple: A tuple containing:
            - woe_df (pd.DataFrame): DataFrame with WoE and IV_Contrib for each category.
            - iv (float): Total Information Value for the feature.
    """
    if not all(data[target].isin([0, 1])):
        raise ValueError(f"Target column '{target}' must contain only 0s and 1s.")
    
    # Ensure the feature is treated as categorical for grouping
    df_temp = data.copy()
    df_temp[feature] = df_temp[feature].astype(str) # Convert to string to ensure correct grouping for all types

    grouped = df_temp.groupby(feature)[target].agg(['count', 'sum'])
    grouped.columns = ['Total', 'Bad']  # 'Bad' refers to the positive class (default)
    grouped['Good'] = grouped['Total'] - grouped['Bad'] # 'Good' refers to the negative class (non-default)
    
    total_good = grouped['Good'].sum()
    total_bad = grouped['Bad'].sum()

    if total_good == 0 or total_bad == 0:
        print(f"Warning: Total good or bad cases are zero for feature '{feature}'. IV will be 0.")
        return pd.DataFrame(), 0.0

    grouped['Good_Pct'] = grouped['Good'] / total_good
    grouped['Bad_Pct'] = grouped['Bad'] / total_bad

    epsilon = 0.000001
    grouped['WoE'] = np.log((grouped['Good_Pct'] + epsilon) / (grouped['Bad_Pct'] + epsilon))
    grouped['IV_Contrib'] = (grouped['Good_Pct'] - grouped['Bad_Pct']) * grouped['WoE']
    iv = grouped['IV_Contrib'].sum()

    woe_df = grouped.reset_index()
    woe_df = woe_df[[feature, 'Good', 'Bad', 'Good_Pct', 'Bad_Pct', 'WoE', 'IV_Contrib']]
    woe_df.rename(columns={feature: 'Category'}, inplace=True)
    woe_df['Feature'] = feature

    return woe_df, iv

In [ ]:
def apply_woe_transformation(data_to_transform, woe_map, feature_name, new_col_name=None):
    """
    Applies WoE transformation to a specified feature in a DataFrame.
    
    Args:
        data_to_transform (pd.DataFrame): The DataFrame to transform.
        woe_map (dict): A dictionary mapping feature categories to their WoE values.
        feature_name (str): The name of the original feature column.
        new_col_name (str, optional): The name for the new WoE transformed column.
                                      Defaults to f'{feature_name}_WoE'.
                                      
    Returns:
        pd.DataFrame: The DataFrame with the new WoE transformed column.
    """
    if new_col_name is None:
        new_col_name = f'{feature_name}_WoE'
    
    temp_series = data_to_transform[feature_name].astype(str)
    
    # Apply the WoE mapping
    data_to_transform[new_col_name] = temp_series.map(woe_map).fillna(0) 
    return data_to_transform

In [ ]:
def calculate_csi(expected, actual):
    """Calculates CSI for two pandas Series."""
    all_categories = pd.concat([expected, actual]).unique()
    expected_dist = expected.value_counts(normalize=True).reindex(all_categories, fill_value=0).sort_index()
    actual_dist = actual.value_counts(normalize=True).reindex(all_categories, fill_value=0).sort_index()

    epsilon = 0.000001
    expected_dist = expected_dist.replace(0, epsilon)
    actual_dist = actual_dist.replace(0, epsilon)

    csi = ((expected_dist - actual_dist) * np.log(expected_dist / actual_dist)).sum()
    return csi

In [ ]:
# Load data
df = pd.read_csv('car.csv', low_memory=False)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.shape()

In [ ]:
# Apply data cleaning
df_cleaned = drop_irrelevant_columns(df.copy())
df_cleaned = define_target_variable(df_cleaned.copy())

In [ ]:
# Drop the original 'loan_status' column after creating 'loan_default'
if 'loan_status' in df_cleaned.columns:
        df_cleaned = df_cleaned.drop(columns=['loan_status'])
    
df_cleaned = handle_missing_values(df_cleaned.copy())

In [ ]:
# Apply feature engineering
df_fe = create_financial_ratios(df_cleaned.copy())
df_fe = extract_date_features(df_fe.copy())
df_fe = convert_string_to_numeric(df_fe.copy())

In [ ]:
# Identify numerical and categorical columns after feature engineering and conversions
numerical_features_all, categorical_features_all = identify_feature_types(df_fe.copy(), exclude_cols=['loan_default'])

# Cap outliers using the updated list of numerical features
df_fe = cap_outliers(df_fe.copy(), numerical_features_all)

In [ ]:
# Separate features (X) and target (y)
X = df_fe.drop('loan_default', axis=1)
y = df_fe['loan_default']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
print(f"\nData split into training (X_train shape: {X_train.shape}) and testing (X_test shape: {X_test.shape}) sets.")
print(f"Target distribution in training: {y_train.value_counts(normalize=True)}")
print(f"Target distribution in testing: {y_test.value_counts(normalize=True)}")

In [ ]:
print("\n--- CSI Report (Before Advanced Preprocessing) ---")
csi_report = []
all_features_for_csi = list(set(X_train.columns) | set(X_test.columns))

for col in all_features_for_csi:
    if col in X_train.columns and col in X_test.columns:
        csi_val = calculate_csi(X_train[col].astype(str), X_test[col].astype(str))
        csi_report.append({'Feature': col, 'CSI': round(csi_val, 4)})
csi_df = pd.DataFrame(csi_report).sort_values(by='CSI', ascending=False)
print(csi_df)
print("\n--- End of CSI Report ---")

In [ ]:
# Drop columns with CSI less than 0.1
cols_to_drop_by_csi = csi_df[csi_df['CSI'] < 0.1]['Feature'].tolist()
X_train = X_train.drop(columns=cols_to_drop_by_csi, errors='ignore')
X_test = X_test.drop(columns=cols_to_drop_by_csi, errors='ignore')
print(f"Dropped {len(cols_to_drop_by_csi)} columns with CSI less than 0.1: {cols_to_drop_by_csi}")
print(f"New X_train shape after CSI-based dropping: {X_train.shape}")
print(f"New X_test shape after CSI-based dropping: {X_test.shape}")

In [ ]:
# --- WoE and IV Calculation ---
print("\n--- Calculating WoE and IV for selected features ---")

# Combine X_train and y_train for WoE calculation
train_for_woe_iv = X_train.copy()
train_for_woe_iv['loan_default'] = y_train

woe_maps = {} 
iv_summary = []
woe_data_dfs = [] 

# Identify features for WoE/IV calculation
features_for_woe_iv_calculation = []

# Add categorical features (excluding high cardinality ones if desired, though current code doesn't filter by cardinality)
for col in categorical_features_all:
    if col in train_for_woe_iv.columns: 
        features_for_woe_iv_calculation.append(col)
        
# Add low-cardinality numerical features
for col in numerical_features_all: 
    if col in train_for_woe_iv.columns and train_for_woe_iv[col].nunique() < 20:
         if col != 'loan_default': 
            features_for_woe_iv_calculation.append(col)

In [ ]:
# Special handling for 'annual_inc': Bin it for WoE calculation
annual_inc_binned_col = 'annual_inc_binned'
annual_inc_bins = None # Initialize to None
if 'annual_inc' in X_train.columns:
    try:
        train_for_woe_iv[annual_inc_binned_col], annual_inc_bins = pd.qcut(train_for_woe_iv['annual_inc'], 
                                                                            q=5, duplicates='drop', 
                                                                            labels=False, # Use integer labels
                                                                            retbins=True)
        X_train[annual_inc_binned_col] = pd.cut(X_train['annual_inc'], bins=annual_inc_bins, 
                                                 labels=False, include_lowest=True, right=True, 
                                                 duplicates='drop')
        features_for_woe_iv_calculation.append(annual_inc_binned_col)
    except Exception as e:
        print(f"Warning: Could not bin 'annual_inc' for WoE calculation: {e}")
        if annual_inc_binned_col in features_for_woe_iv_calculation:
            features_for_woe_iv_calculation.remove(annual_inc_binned_col)

In [ ]:
# Clean up features_for_woe_iv_calculation to remove potential duplicates or problematic columns
features_for_woe_iv_calculation = list(set(features_for_woe_iv_calculation))

# Perform WoE/IV calculation
for feature in features_for_woe_iv_calculation:
    print(f"Calculating WoE and IV for: {feature}")
    try:
        woe_df, iv = calculate_woe_iv(train_for_woe_iv, feature, 'loan_default')
        iv_summary.append({'Feature': feature, 'IV': iv})
        if not woe_df.empty:
            woe_data_dfs.append(woe_df)
            # Store WoE mapping for later transformation
            woe_map = dict(zip(woe_df['Category'].astype(str), woe_df['WoE']))
            woe_maps[feature] = woe_map
        print(f"  IV for {feature}: {iv:.4f}")
    except ValueError as e:
        print(f"  Error calculating WoE/IV for {feature}: {e}")
    except Exception as e:
        print(f"  An unexpected error occurred for {feature}: {e}")

if iv_summary:
    iv_summary_df = pd.DataFrame(iv_summary).sort_values(by='IV', ascending=False)
    print("\n--- Information Value (IV) Summary ---")
    print(iv_summary_df)

    print("\n--- WoE Details for each Feature ---")
    if woe_data_dfs:
        full_woe_df = pd.concat(woe_data_dfs, ignore_index=True)
        print(full_woe_df)
    else:
        print("No WoE details generated.")
else:
    print("No IV summary available.")

In [ ]:
# --- Apply WoE Transformation ---
print("\n--- Applying WoE Transformation ---")
woe_transformed_cols = []

# Handle annual_inc binning for X_test before applying WoE, using bins from X_train
if 'annual_inc' in X_test.columns and annual_inc_binned_col in features_for_woe_iv_calculation and annual_inc_bins is not None:
    X_test[annual_inc_binned_col] = pd.cut(X_test['annual_inc'], bins=annual_inc_bins, 
                                             labels=False, include_lowest=True, right=True, 
                                             duplicates='drop')
    X_test[annual_inc_binned_col].fillna(X_train[annual_inc_binned_col].mode()[0], inplace=True)
    X_train[annual_inc_binned_col].fillna(X_train[annual_inc_binned_col].mode()[0], inplace=True)


for feature, woe_map in woe_maps.items():
    # Apply WoE to training set
    X_train = apply_woe_transformation(X_train, woe_map, feature)
    # Apply WoE to test set
    X_test = apply_woe_transformation(X_test, woe_map, feature)
    woe_transformed_cols.append(f'{feature}_WoE')
    print(f"  Applied WoE for '{feature}'")

In [ ]:
# --- Update Feature Lists for ColumnTransformer ---
# Numerical features now include the new WoE transformed columns
numerical_features_for_pipeline = [col for col in numerical_features_all if col not in features_for_woe_iv_calculation] + woe_transformed_cols

categorical_features_for_pipeline = [col for col in categorical_features_all if col not in features_for_woe_iv_calculation]

X_train = X_train.drop(columns=features_for_woe_iv_calculation, errors='ignore')
X_test = X_test.drop(columns=features_for_woe_iv_calculation, errors='ignore')

numerical_features_for_pipeline = [col for col in numerical_features_for_pipeline if col in X_train.columns]
categorical_features_for_pipeline = [col for col in categorical_features_for_pipeline if col in X_train.columns]

print(f"\nFeatures for StandardScaler: {numerical_features_for_pipeline}")
print(f"Features for OneHotEncoder: {categorical_features_for_pipeline}")

In [ ]:
preprocessor = create_preprocessing_pipeline(numerical_features_for_pipeline, categorical_features_for_pipeline)

X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

transformed_feature_names = []
transformed_feature_names.extend(numerical_features_for_pipeline)

# if 'cat' in [name for name, _, _ in preprocessor.transformers]:
#     # Access the fitted OneHotEncoder from the preprocessor's transformers_
#     ohe_transformer_index = [i for i, (name, _, _) in enumerate(preprocessor.transformers_) if name == 'cat'][0]
#     ohe_transformer = preprocessor.transformers_[ohe_transformer_index][1]
#     ohe_feature_names = ohe_transformer.get_feature_names_out(categorical_features_for_pipeline)
#     transformed_feature_names.extend(list(ohe_feature_names))

X_train_df_final = pd.DataFrame(X_train_processed, index=X_train.index)
X_test_df_final = pd.DataFrame(X_test_processed, index=X_test.index)

print("\nApplying SMOTE to training data...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_df_final, y_train)

In [ ]:
print("\n--- Training and Evaluating Multiple Models ---")

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, solver='liblinear', random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=100),
    "SVC (Linear Kernel)": SVC(kernel='linear', probability=True, random_state=42, max_iter=1000), # probability=True for ROC AUC
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "XGBoost": xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
}

results = {}

for name, model in models.items():
    print(f"\n--- Training {name} ---")
    try:
        model.fit(X_train_resampled, y_train_resampled)
        print(f"{name} trained successfully.")

        y_pred_test = model.predict(X_test_df_final)
        test_acc = accuracy_score(y_test, y_pred_test) * 100
        
        print(f"\n--- {name} Test Set Evaluation ---")
        print(f"Test Accuracy: {test_acc:.2f}%")
        print("Classification Report (Test):\n", classification_report(y_test, y_pred_test))
        print("Confusion Matrix (Test):\n", confusion_matrix(y_test, y_pred_test))

        if hasattr(model, "predict_proba"):
            y_proba_test = model.predict_proba(X_test_df_final)[:, 1]
            test_roc_auc = roc_auc_score(y_test, y_proba_test)
            print(f"ROC AUC Score (Test): {test_roc_auc:.4f}")

            fpr, tpr, _ = roc_curve(y_test, y_proba_test)
            plt.figure(figsize=(8, 6))
            plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {test_roc_auc:.2f})')
            plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.title(f'ROC Curve for {name}')
            plt.legend(loc="lower right")
            plt.grid(True)
            plt.show()
        else:
            print(f"ROC AUC Score not available for {name} (model does not have predict_proba).")
        
        results[name] = {
            "Accuracy": test_acc,
            "Classification Report": classification_report(y_test, y_pred_test, output_dict=True),
            "Confusion Matrix": confusion_matrix(y_test, y_pred_test).tolist(),
            "ROC AUC": test_roc_auc if hasattr(model, "predict_proba") else "N/A"
        }

    except Exception as e:
        print(f"An error occurred during training/evaluation of {name}: {e}")
        results[name] = {"Error": str(e)}

In [ ]:
print("\n--- All Models Evaluation Summary ---")
for name, res in results.items():
    print(f"\nModel: {name}")
    if "Error" in res:
        print(f"  Error: {res['Error']}")
    else:
        print(f"  Test Accuracy: {res['Accuracy']:.2f}%")
        print(f"  Test ROC AUC: {res['ROC AUC']:.4f}" if res['ROC AUC'] != "N/A" else f"  Test ROC AUC: {res['ROC AUC']}")
        print(f"  Test Precision (Class 1): {res['Classification Report']['1']['precision']:.2f}")
        print(f"  Test Recall (Class 1): {res['Classification Report']['1']['recall']:.2f}")